# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,291.31,88,20,5.14,ZA,1628156890
1,1,ancud,-41.8697,-73.8203,283.89,81,100,9.75,CL,1628157038
2,2,ushuaia,-54.8000,-68.3000,281.96,66,20,2.06,AR,1628157038
3,3,maarianhamina,60.0973,19.9348,292.07,68,0,5.66,AX,1628157038
4,4,tarauaca,-8.1614,-70.7656,291.68,62,7,1.13,BR,1628157039


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
#configure the map
gmaps.configure(api_key=g_key)

In [19]:
#Heatmap of Humidtiy
location = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
# Trim down city list drop null values
reduce_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 300) & (city_data_df["Max Temp"] > 294)\
                                 & (city_data_df["Wind Speed"] < 10) \
                                 & (city_data_df["Cloudiness"] == 0)].dropna()
reduce_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,carnarvon,-24.8667,113.6333,298.19,35,0,6.69,AU,1628157044
92,92,namibe,-15.1961,12.1522,294.50,78,0,0.00,AO,1628157079
122,122,tiznit,29.5833,-9.5000,299.09,49,0,2.24,MA,1628157092
189,189,aprelevka,55.5519,37.0801,297.59,49,0,3.00,RU,1628157120
404,404,dondo,-19.6094,34.7431,297.88,41,0,2.07,MZ,1628157211
417,417,chadiza,-14.0678,32.4392,299.14,21,0,1.81,ZM,1628157217
432,432,medea,36.2642,2.7539,297.40,20,0,2.40,DZ,1628157223
444,444,mkushi,-13.6202,29.3939,299.52,18,0,1.92,ZM,1628157228
462,462,letlhakane,-21.4167,25.5833,296.96,21,0,3.38,BW,1628157236
480,480,belyy yar,53.6039,91.3903,299.09,44,0,3.00,RU,1628157245


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
#create a dataframe to store hotel and city data
hotel_df = reduce_city_df[["City","Country","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
15,carnarvon,AU,-24.8667,113.6333,
92,namibe,AO,-15.1961,12.1522,
122,tiznit,MA,29.5833,-9.5000,
189,aprelevka,RU,55.5519,37.0801,
404,dondo,MZ,-19.6094,34.7431,
417,chadiza,ZM,-14.0678,32.4392,
432,medea,DZ,36.2642,2.7539,
444,mkushi,ZM,-13.6202,29.3939,
462,letlhakane,BW,-21.4167,25.5833,
480,belyy yar,RU,53.6039,91.3903,


In [30]:
#write parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params).json()
    
    try:
        hote_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except(KeyError, IndexError):
        print("Missing field/result...skipping")
hotel_df

Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping


,City,Country,Lat,Lng,Hotel Name
15,carnarvon,AU,-24.8667,113.6333,
92,namibe,AO,-15.1961,12.1522,
122,tiznit,MA,29.5833,-9.5000,
189,aprelevka,RU,55.5519,37.0801,
404,dondo,MZ,-19.6094,34.7431,
417,chadiza,ZM,-14.0678,32.4392,
432,medea,DZ,36.2642,2.7539,
444,mkushi,ZM,-13.6202,29.3939,
462,letlhakane,BW,-21.4167,25.5833,
480,belyy yar,RU,53.6039,91.3903,


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))